# EEG Statistics

Using some of the data you collected, we are going to go through a couple of different ways to discern whether the difference in signal between your conditions is statistically significant.

#### Setting up Python
Before starting to analyse our own EEG data, we need to make sure we have our virtual environment we created during the `MNE-tutorial`.

1. Press `Select Kernel`, then `Python Environments...` and then choose any Python kernel. 
2. Run the code chunk below
3. Change the kernel used to run the code in this notebook. Press where it says `Python X.XX.XX` in the top right corner, then `Select Another Kernel`, then `Jupyter kernel...` and then select `env`. If `env` does not show up, press the little refresh symbol! 

In [ ]:
!bash ../env_to_ipynb_kernel.sh

#### Import packages

In [4]:
import mne
import numpy as np
import pandas as pd
from pathlib import Path

# importing Lauras homemade functions for preprocessing
import sys
sys.path.append("..")
from helper_functions import preproc_subject, update_event_ids, update_events_group1_group2, event_id


## Epochs
We need epochs for the statistical tests. We will be using the function from earlier today to preprocess the data!

In [ ]:
# Load raw data 
# reading the file & loading in the data
# path to the data folder 
data_folder = Path("/work/EEG_lab/raw")
group_number= "1"

# path to the data - MAKE SURE TO CHANGE PATH ACCORDING TO YOUR GROUP NUMBER!
data_path = data_folder / "EEG" / f"group{group_number}.vhdr"

raw = mne.io.read_raw_brainvision(data_path)
raw.load_data()

# reading in the csv file with experiment information
behavioural_path = data_folder / "behavioural" / f"subject-{group_number}.csv"
logfile = pd.read_csv(behavioural_path)
logfile.head() # prints the first lines of the csv file

# clean logfile
logfile["target_gender"] = logfile["target_gender"].apply(lambda x: x.strip())
logfile["target_gender"] = logfile["target_gender"].apply(lambda x: x.strip("??"))

events, _ = mne.events_from_annotations(raw)


The preprocessing function needs to know which channels are bad. Please insert the ones you found yesterday here!

In [6]:
# INSERT BAD CHANNELS
bad_channels = []

In [7]:
# ONLY GROUP 1 and GROUP 2
# get rid of practise trials from the logfile
if group_number in ["1", "2"]: # checks the group number before it runs the code
    logfile = logfile[logfile["practice"]=="no"]
    events = update_events_group1_group2(events, event_id, logfile)

In [ ]:
# Lets preprocess the data
epochs = preproc_subject(
    raw, 
    event_id = update_event_ids(events, event_id), 
    events = events,
    bad_channels = bad_channels, 
    reject = {"eeg":150e-6}, 
    l_freq = 1, 
    h_freq = 40, 
    tmin = -0.2, 
    tmax = 0.5, 
    baseline = (None, 0)
    )

Okay, now it is time for you to make some decisions! At this point you need to choose a contrast:
1. 'word/target/female' & 'word/target/male'

2. 'word/congruent' & 'word/incongruent'

3. 'word/prime' & 'word/target'


In [14]:
# Extract the contrast you are interested in using the following code as an example
epochs_condition1 = epochs["word/target/female"]
epochs_condition2 = epochs["word/target/female"]

In [ ]:
# lets check which events were included
print(epochs_condition1)
print(epochs_condition2)

## Windowed mean
Now we have our two conditions: trials with words vs images. One of the simplest way in which we can determine whether the signal in our two conditions are statistically significant is by:

1) Segmenting our data using only certain channels in a specific time window. Keep in mind that which time window and channels should be established a priori, for instance according to the literature. 
2) Taking the mean of that window across channels and and samples.
3) Running statistical tests on the windowed means from the two conditions.

In an experiment with multiple participants we would also average over trials from individual participants, in order to only have one data point per participant (and thereby avoid multiple comparisons). However, since we have one participant, we can keep one dimension of the individual data, i.e. the trials.

### T-test
We can now do a t-test on the trials from the two conditions, to establish whether the means of the two groups are statistically significant.

We can use the get_data() function to get the numerical values of the signal (in microvolts) for the t-test. tmin and tmax are used to define the size of the window, and the picks are the channels that we expect to see an effect in.

In [ ]:
# INSERT CHANNEL NAMES YOU ARE INTERESTED IN BELOW
picks = ["xxx", "xxx", "xxx"]


# DETERMINE THE TIME FRAME YOU WANT TO LOOK AT (remember to write it in seconds )
tmin_ttest = # insert number here
tmax_ttest = # insert number here


In [ ]:
# Now we can extract the data using the following logic
data_condition1 = epochs_condition1.get_data(picks = picks, tmin = tmin_ttest, tmax = tmax_ttest)
data_condition2 = epochs_condition2.get_data(picks = picks, tmin = tmin_ttest, tmax = tmax_ttest)

Investigating the resulting data; how many dimensions does the data have? What do you think they represent (i.e. which dimension is channels, trials, etc.)?

In [ ]:
print(data_condition1.shape)
print(data_condition2.shape)

Now we can average over the data so we only have the trials dimension.

In [ ]:
data_condition1_mean = np.mean(data_condition1, axis=2) # averaging over the third dimension of the data
print(data_condition1_mean.shape)

data_condition1_mean = np.mean(data_condition1_mean, axis=1) # averaging over the second dimension of the data
print(data_condition1_mean.shape)

## Now do the same for the second condition!

# INSERT CODE HERE

### Running the t-test

In [3]:
# installing additional packages
from scipy import stats as st
import statistics as stats

In [ ]:
# running the t-test
st.ttest_ind(a=data_condition1_mean, b=data_condition2_mean)

### Creating a nice plot to go with the t-test


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, figsize=(10, 5), dpi=300)

# plot the time window used for the t-test
ax.axvspan(tmin, tmax, color="grey", alpha=0.2, label = "Time window for t-test")

data_condition1_plot = epochs_condition1.get_data(picks = picks).mean(axis=(1,2)) # also extracting the data outside of the time window to show the entire epoch duration
data_condition2_plot = epochs_condition2.get_data(picks = picks).mean(axis=(1,2))

# GET TIMES LAURA

ax.plot(times, data_condition1_plot, label="Nonword", linestyle="-", color="orange")
ax.plot(times, data_condition2_plot, label="Word", linestyle="-", color="blue")

# vertical line at 0
ax.axvline(x=0, color="black", linestyle="--", linewidth=1)

ax.set(xlabel="Time (s)", ylabel="Amplitude", title="ERP (INSERT CHOSEN CHANNEL NAMES HERE) ")
ax.legend()
plt.savefig("plots/figure1.png")